In [ ]:
# Import der Bibliotheken

import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import date, datetime
import plotly

In [ ]:
# Konfiguration der Grafiken

plt.rcParams['savefig.dpi'] = 200
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.autolayout'] = False

plt.rcParams['figure.figsize'] = (20, 10)
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 14
plt.rcParams['legend.fontsize'] = 12

plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['lines.markersize'] = 8

plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14


# Definition von Funktionen

In [ ]:
def Jahresverlauf(Titel,Anfang,Ende,Dataframe,Stunde,xlabel,ylabel,h_offset,v_offset):
    
    df = pd.DataFrame(
        {'Hours': pd.date_range('2025-01-01', '2026-01-01', freq='1H', closed='left')}
     )
    Dataframe1=Dataframe.copy()
    Dataframe1[Stunde]=df.Hours
    Dataframe1=Dataframe1.set_index(Stunde)
    Dataframe1=Dataframe1.loc[Anfang:Ende]
    fig = plt.figure(figsize=(19,8))
 
    ax1 = fig.add_axes([1, 1, 1, 1])
    ax1.set_title(Titel,weight='bold',fontsize=20)
    ax1.plot(Dataframe1.index,Dataframe1)
    colum_names=[]
    column_names = list(Dataframe1.columns.values)
    plt.legend(labels= column_names)
    plt.xlabel(xlabel,fontsize=15)
    plt.ylabel(ylabel,fontsize=15)
    horiz_offset = h_offset
    vert_offset = v_offset
    ax1.legend(labels=column_names,bbox_to_anchor=(horiz_offset, vert_offset))
 
    plt.show()

In [ ]:
# Funktion zur Abbildung der Jahresdauerlinie

def Dauerlinie (Dataframe, Column, Titel,Yachse ):
  df_sort = Dataframe.sort_values(by=[Column], ascending=False) # sortieren der Wirkleistungen in absteigender Reihenfolge
  df_sort = df_sort.reset_index(drop=True) #zurücksetzen des Index
  plt.plot(df_sort.index, df_sort[Column]) # Plotten der Dauerlinie
  plt.title(Titel)
  plt.ylabel(Yachse)

In [ ]:
# Auswahl des Standortes
location_north = False

# Einlesen der Datenreihen

In [ ]:
# Erzeugungsprofile

if location_north:
    data_pv = pd.read_csv('data/data_pv_antofagasta_1MW.csv', skiprows = 3)
    data_wind = pd.read_csv('data/data_wind_antofagasta_1MW.csv', skiprows = 3)
    
else:
    data_pv = pd.read_csv('data/data_pv_punta_arenas_1MW.csv', skiprows = 3)
    data_wind = pd.read_csv('data/data_wind_punta_arenas_1MW.csv', skiprows = 3)
    
data_shipping = pd.read_excel('data/Ferntransport.xlsx', usecols = (0,1))

In [ ]:
# Erstellung eines Dataframes aus externen Daten

dataset_df=pd.DataFrame()
dataset_df['Zeitstempel']=data_pv['time']
dataset_df['PV_Erzeugung']=data_pv['electricity']
dataset_df['Wind_Erzeugung']=data_wind['electricity']

if location_north:
    dataset_df['Kapazität_Schiff']=data_shipping['Antofagasta']
else:
    dataset_df['Kapazität_Schiff']=data_shipping['Punta Arenas']
    
dataset_df.Kapazität_Schiff=dataset_df.Kapazität_Schiff.fillna(method='ffill')

In [ ]:
# Einlesen und Ausgabe der Inputs für die Variablen

data_inputs = pd.read_excel('data/Inputs_Modell.xlsx', sheet_name = 'Gesamt', header = 2, index_col = 0, nrows = 19)

data_inputs=pd.DataFrame(data_inputs.fillna(0)).drop(columns = ['Unnamed: 1','name','bus','p_nom','Unnamed: 10'])
data_inputs

# Definition der Variablen

In [ ]:
# Erstellung der Last durch den Wasserstoffbedarf

hydrogen_energy_content = 33 #kWh/kg oder MWh/t
hydrogen_demand_t = 35 * 1000
hydrogen_demand_kg = hydrogen_demand_t * 1000
hydrogen_demand_MWh = hydrogen_demand_t * hydrogen_energy_content # MWh
hydrogen_demand_hourly = hydrogen_demand_MWh/8760 #MW

print('Der stündliche Bedarf bei konstanter Verteilung des Jahresbedarfs von 35 kt Wasserstoff beträgt ungefähr ' + str(round(hydrogen_demand_hourly,0)) + ' MWh')

## Investitionskosten

In [ ]:
# capital_cost

# Erzeugung
capital_cost_pv = data_inputs.loc['Photovoltaik (Chile)']['capital_cost'] # €/MW
capital_cost_wind = data_inputs.loc['Windkraft (Chile)']['capital_cost'] # €/MW


# Elektrolyse
capital_cost_PEMEL = data_inputs.loc['PEM Elektrolyse']['capital_cost'] # €/MW
capital_cost_AEL = data_inputs.loc['AEL Elektrolyse']['capital_cost'] # €/MW


# Umwandlung
capital_cost_LH2 = data_inputs.loc['Verflüssigung']['capital_cost'] # €/MW
capital_cost_NH3 = data_inputs.loc['Haber-Bosch']['capital_cost'] # €/MW
capital_cost_LOHC = data_inputs.loc['Hydrierung']['capital_cost'] # €/MW
capital_cost_LH2_b = data_inputs.loc['Regasifizierung']['capital_cost'] # €/MW
capital_cost_NH3_b = data_inputs.loc['NH3 Cracker']['capital_cost'] # €/MW
capital_cost_LOHC_b = data_inputs.loc['Dehydrierung']['capital_cost'] # €/MW


#Transport
capital_cost_pipeline = 0 
capital_cost_shipping_LH2 = data_inputs.loc['LH2 Schiff']['capital_cost'] # €/MW
capital_cost_shipping_NH3 = data_inputs.loc['NH3 Schiff']['capital_cost'] # €/MW
capital_cost_shipping_LOHC = data_inputs.loc['LOHC Schiff']['capital_cost'] # €/MW


# Speicherung
capital_cost_storage_el = data_inputs.loc['Stromspeicher']['capital_cost'] *1000
capital_cost_storage_LH2 = data_inputs.loc['LH2 Speicher']['capital_cost'] # €/MW
capital_cost_storage_NH3 = data_inputs.loc['NH3 Speicher']['capital_cost'] # €/MW
capital_cost_storage_LOHC = data_inputs.loc['LOHC Speicher']['capital_cost'] # €/MW

## Gestehungskosten

In [ ]:
# marginal_cost

# Erzeugung
marginal_cost_pv = data_inputs.loc['Photovoltaik (Chile)']['marginal_cost'] # €/MWh
marginal_cost_wind = data_inputs.loc['Windkraft (Chile)']['marginal_cost'] # €/MWh

# Elektrolyse
marginal_cost_PEMEL = data_inputs.loc['PEM Elektrolyse']['marginal_cost'] # €/MWh
marginal_cost_AEL = data_inputs.loc['AEL Elektrolyse']['marginal_cost'] # €/MWh

# Umwandlung
marginal_cost_LH2 = data_inputs.loc['Verflüssigung']['marginal_cost'] # €/MWh
marginal_cost_NH3 = data_inputs.loc['Haber-Bosch']['marginal_cost'] # €/MWh
marginal_cost_LOHC = data_inputs.loc['Hydrierung']['marginal_cost'] # €/MWh
marginal_cost_LH2_b = data_inputs.loc['Regasifizierung']['marginal_cost'] # €/MWh
marginal_cost_NH3_b = data_inputs.loc['NH3 Cracker']['marginal_cost'] # €/MWh
marginal_cost_LOHC_b = data_inputs.loc['Dehydrierung']['marginal_cost'] # €/MWh

# Transport
marginal_cost_pipeline = data_inputs.loc['Pipeline']['marginal_cost'] # €/MWh
marginal_cost_shipping_LH2 = data_inputs.loc['LH2 Schiff']['marginal_cost'] # €/MWh
marginal_cost_shipping_NH3 = data_inputs.loc['NH3 Schiff']['marginal_cost'] # €/MWh
marginal_cost_shipping_LOHC = data_inputs.loc['LOHC Schiff']['marginal_cost'] # €/MWh

# Speicherung
marginal_cost_storage_el = data_inputs.loc['Stromspeicher']['marginal_cost'] # €/MWh
marginal_cost_storage_LH2 = data_inputs.loc['LH2 Speicher']['marginal_cost'] # €/MWh
marginal_cost_storage_NH3 = data_inputs.loc['NH3 Speicher']['marginal_cost'] # €/MWh
marginal_cost_storage_LOHC = data_inputs.loc['LOHC Speicher']['marginal_cost'] # €/MWh

## Effizienz

In [ ]:
# efficiency

# Wirkungsgrad Erzeugung
efficiency_PEMEL = data_inputs.loc['PEM Elektrolyse']['efficiency']
efficiency_AEL = data_inputs.loc['AEL Elektrolyse']['efficiency']

# Wirkungsgrad Umwandlung
efficiency_LH2 = data_inputs.loc['Verflüssigung']['efficiency']
efficiency_NH3 = data_inputs.loc['Haber-Bosch']['efficiency']
efficiency_LOHC = data_inputs.loc['Hydrierung']['efficiency']

efficiency_LH2_b = data_inputs.loc['Regasifizierung']['efficiency']
efficiency_NH3_b = data_inputs.loc['NH3 Cracker']['efficiency']
efficiency_LOHC_b = data_inputs.loc['Dehydrierung']['efficiency']

# Wirkungsgrad der Transporte
efficiency_pipeline = data_inputs.loc['Pipeline']['efficiency']
efficiency_shipping_LH2 = data_inputs.loc['LH2 Schiff']['efficiency'] 
efficiency_shipping_NH3 = data_inputs.loc['NH3 Schiff']['efficiency']
efficiency_shipping_LOHC = data_inputs.loc['LOHC Schiff']['efficiency']


# Speicherverluste
standing_loss_el = 0 #data_inputs.loc['Stromspeicher']['standing_loss']
standing_loss_LH2 = data_inputs.loc['LH2 Speicher']['standing_loss']
standing_loss_NH3 = data_inputs.loc['NH3 Speicher']['standing_loss']
standing_loss_LOHC = data_inputs.loc['LOHC Speicher']['standing_loss']

## Lebenszeit

In [ ]:
# lifetime

lifetime_pv = data_inputs.loc['Photovoltaik (Chile)']['lifetime']
lifetime_wind = data_inputs.loc['Windkraft (Chile)']['lifetime']

lifetime_PEMEL = data_inputs.loc['PEM Elektrolyse']['lifetime']
lifetime_AEL = data_inputs.loc['AEL Elektrolyse']['lifetime']

lifetime_LH2 = data_inputs.loc['Verflüssigung']['lifetime']
lifetime_NH3 = data_inputs.loc['Haber-Bosch']['lifetime']
lifetime_LOHC = data_inputs.loc['Hydrierung']['lifetime']

lifetime_LH2_b = data_inputs.loc['Regasifizierung']['lifetime']
lifetime_NH3_b = data_inputs.loc['NH3 Cracker']['lifetime']
lifetime_LOHC_b = data_inputs.loc['Dehydrierung']['lifetime']

lifetime_shipping_LH2 = data_inputs.loc['LH2 Schiff']['lifetime'] 
lifetime_shipping_NH3 = data_inputs.loc['NH3 Schiff']['lifetime']
lifetime_shipping_LOHC = data_inputs.loc['LOHC Schiff']['lifetime']

lifetime_pipeline = 1

lifetime_storage_el = data_inputs.loc['Stromspeicher']['lifetime']
lifetime_storage_LH2 = data_inputs.loc['LH2 Speicher']['lifetime']
lifetime_storage_NH3 = data_inputs.loc['NH3 Speicher']['lifetime']
lifetime_storage_LOHC = data_inputs.loc['LOHC Speicher']['lifetime']

# Netzwerke aufbauen

In [ ]:
#Erzeugung der drei Netzwerke

network1 = pypsa.Network()
network2 = pypsa.Network()
network3 = pypsa.Network()

# Einstellung der Snapshots

network1.set_snapshots(range(8760))
network2.set_snapshots(range(8760))
network3.set_snapshots(range(8760))

## Buses

In [ ]:
# Erzeugung der Knotenpunkte

# Der Suffix _b steht für Bremerhaven
# Der Suffix _c steht für Chile
    
network1.add('Bus', name = 'electricity')
network1.add('Bus', name = 'GH2_b')
network1.add('Bus', name = 'GH2_c')
network1.add('Bus', name = 'cologne')
network1.add('Bus', name = 'LH2_b')
network1.add('Bus', name = 'LH2_c')

network2.add('Bus', name = 'electricity')
network2.add('Bus', name = 'GH2_b')
network2.add('Bus', name = 'GH2_c')
network2.add('Bus', name = 'cologne')
network2.add('Bus', name = 'NH3_b')
network2.add('Bus', name = 'NH3_c')

network3.add('Bus', name = 'electricity')
network3.add('Bus', name = 'GH2_b')
network3.add('Bus', name = 'GH2_c')
network3.add('Bus', name = 'cologne')
network3.add('Bus', name = 'LOHC_b')
network3.add('Bus', name = 'LOHC_c')

## Loads

In [ ]:
network1.add('Load', name = 'hydrogen_demand', bus = 'cologne', 
            p_set = hydrogen_demand_hourly)

network2.add('Load', name = 'hydrogen_demand', bus = 'cologne', 
            p_set = hydrogen_demand_hourly)

network3.add('Load', name = 'hydrogen_demand', bus = 'cologne', 
            p_set = hydrogen_demand_hourly)

## Generators

In [ ]:
network1.add('Generator', name = 'Photovoltaik', bus = 'electricity', 
             p_max_pu = dataset_df['PV_Erzeugung']/dataset_df['PV_Erzeugung'].max(), 
             p_min_pu = dataset_df['PV_Erzeugung']/dataset_df['PV_Erzeugung'].max(), 
             p_nom_extendable = True,
             marginal_cost = marginal_cost_pv, 
             capital_cost = capital_cost_pv,
            lifetime = lifetime_pv)
network1.add('Generator', name = 'Windkraft', bus = 'electricity',
             p_max_pu = dataset_df['Wind_Erzeugung']/dataset_df['Wind_Erzeugung'].max(), 
             p_min_pu = dataset_df['Wind_Erzeugung']/dataset_df['Wind_Erzeugung'].max(), 
             p_nom_extendable = True,
             marginal_cost = marginal_cost_wind, 
             capital_cost = capital_cost_wind,
            lifetime = lifetime_wind)

network2.add('Generator', name = 'Photovoltaik', bus = 'electricity', 
             p_max_pu = dataset_df['PV_Erzeugung']/dataset_df['PV_Erzeugung'].max(), 
             p_min_pu = dataset_df['PV_Erzeugung']/dataset_df['PV_Erzeugung'].max(), 
             p_nom_extendable = True,
             marginal_cost = marginal_cost_pv, 
             capital_cost = capital_cost_pv,
            lifetime = lifetime_pv)
network2.add('Generator', name = 'Windkraft', bus = 'electricity',
             p_max_pu = dataset_df['Wind_Erzeugung']/dataset_df['Wind_Erzeugung'].max(), 
             p_min_pu = dataset_df['Wind_Erzeugung']/dataset_df['Wind_Erzeugung'].max(), 
             p_nom_extendable = True,
             marginal_cost = marginal_cost_wind, 
             capital_cost = capital_cost_wind,
            lifetime = lifetime_wind)

network3.add('Generator', name = 'Photovoltaik', bus = 'electricity', 
             p_max_pu = dataset_df['PV_Erzeugung']/dataset_df['PV_Erzeugung'].max(), 
             p_min_pu = dataset_df['PV_Erzeugung']/dataset_df['PV_Erzeugung'].max(), 
             p_nom_extendable = True,
             marginal_cost = marginal_cost_pv, 
             capital_cost = capital_cost_pv,
            lifetime = lifetime_pv)
network3.add('Generator', name = 'Windkraft', bus = 'electricity',
             p_max_pu = dataset_df['Wind_Erzeugung']/dataset_df['Wind_Erzeugung'].max(), 
             p_min_pu = dataset_df['Wind_Erzeugung']/dataset_df['Wind_Erzeugung'].max(), 
             p_nom_extendable = True,
             marginal_cost = marginal_cost_wind, 
             capital_cost = capital_cost_wind,
            lifetime = lifetime_wind)

## Links - Electrolysis

In [ ]:
network1.add('Link', name = 'PEMEL', bus0 = 'electricity', bus1 = 'GH2_c',
                p_nom_extendable = True,
                lifetime = lifetime_PEMEL,
                marginal_cost = marginal_cost_PEMEL, 
                capital_cost = capital_cost_PEMEL,
                efficiency = efficiency_PEMEL)
network1.add('Link', name = 'AEL', bus0 = 'electricity', bus1 = 'GH2_c',
                p_nom_extendable = True,
                lifetime = lifetime_AEL,
                marginal_cost = marginal_cost_AEL, 
                capital_cost = capital_cost_AEL,
                efficiency = efficiency_AEL)

network2.add('Link', name = 'PEMEL', bus0 = 'electricity', bus1 = 'GH2_c',
                p_nom_extendable = True,
                lifetime = lifetime_PEMEL,
                marginal_cost = marginal_cost_PEMEL, 
                capital_cost = capital_cost_PEMEL,
                efficiency = efficiency_PEMEL)    
network2.add('Link', name = 'AEL', bus0 = 'electricity', bus1 = 'GH2_c',
                p_nom_extendable = True,
                lifetime = lifetime_AEL,
                marginal_cost = marginal_cost_AEL, 
                capital_cost = capital_cost_AEL,
                efficiency = efficiency_AEL)

network3.add('Link', name = 'PEMEL', bus0 = 'electricity', bus1 = 'GH2_c',
                p_nom_extendable = True,
                lifetime = lifetime_PEMEL,
                marginal_cost = marginal_cost_PEMEL, 
                capital_cost = capital_cost_PEMEL,
                efficiency = efficiency_PEMEL)    
network3.add('Link', name = 'AEL', bus0 = 'electricity', bus1 = 'GH2_c',
                p_nom_extendable = True,
                lifetime = lifetime_AEL,
                marginal_cost = marginal_cost_AEL, 
                capital_cost = capital_cost_AEL,
                efficiency = efficiency_AEL)

## Links - Conversion

In [ ]:
network1.add('Link', name = 'Verflüssigung', bus0 = 'GH2_c', bus1 = 'LH2_c',
             p_nom_extendable = True,
             marginal_cost = marginal_cost_LH2, 
             capital_cost = capital_cost_LH2,
             efficiency = efficiency_LH2,
             lifetime = lifetime_LH2)
network1.add('Link', name = 'Regasifizierung', bus0 = 'LH2_b', bus1 = 'GH2_b',
             p_nom_extendable = True,
             marginal_cost = marginal_cost_LH2_b, 
             capital_cost = capital_cost_LH2_b,
             efficiency = efficiency_LH2_b,
             lifetime = lifetime_LH2_b)

network2.add('Link', name = 'Haber-Bosch', bus0 = 'GH2_c', bus1 = 'NH3_c',
             p_nom_extendable = True,
             marginal_cost = marginal_cost_NH3, 
             capital_cost = capital_cost_NH3,
             efficiency = efficiency_NH3,
             lifetime = lifetime_NH3)
network2.add('Link', name = 'NH3 Cracker', bus0 = 'NH3_b', bus1 = 'GH2_b',
             p_nom_extendable = True,
             marginal_cost = marginal_cost_NH3_b, 
             capital_cost = capital_cost_NH3_b,
             efficiency = efficiency_NH3_b,
             lifetime = lifetime_NH3_b)

network3.add('Link', name = 'Hydrierung', bus0 = 'GH2_c', bus1 = 'LOHC_c',
             p_nom_extendable = True,
             marginal_cost = marginal_cost_LOHC, 
             capital_cost = capital_cost_LOHC,
             efficiency = efficiency_LOHC,
             lifetime = lifetime_LOHC)
network3.add('Link', name = 'Dehydrierung', bus0 = 'LOHC_b', bus1 = 'GH2_b',
             p_nom_extendable = True,
             marginal_cost = marginal_cost_LOHC_b, 
             capital_cost = capital_cost_LOHC_b,
             efficiency = efficiency_LOHC_b,
             lifetime = lifetime_LOHC_b)

## Links - Cargoship

In [ ]:
network1.add('Link', name = 'Verschiffung LH2', bus0 = 'LH2_c', bus1 = 'LH2_b',
             p_nom = dataset_df['Kapazität_Schiff'].max(),
             p_max_pu = dataset_df['Kapazität_Schiff']/dataset_df['Kapazität_Schiff'].max(), 
             p_min_pu = dataset_df['Kapazität_Schiff']/dataset_df['Kapazität_Schiff'].max(),
             p_nom_extendable = True,
             marginal_cost = marginal_cost_shipping_LH2, 
             capital_cost = capital_cost_shipping_LH2,
             efficiency = efficiency_shipping_LH2)

network2.add('Link', name = 'Verschiffung NH3', bus0 = 'NH3_c', bus1 = 'NH3_b',
             p_nom = dataset_df['Kapazität_Schiff'].max(),
             p_max_pu = dataset_df['Kapazität_Schiff']/dataset_df['Kapazität_Schiff'].max(), 
             p_min_pu = dataset_df['Kapazität_Schiff']/dataset_df['Kapazität_Schiff'].max(),
             p_nom_extendable = True,
             marginal_cost = marginal_cost_shipping_NH3, 
             capital_cost = capital_cost_shipping_NH3,
             efficiency = efficiency_shipping_NH3)
    
network3.add('Link', name = 'Verschiffung LOHC', bus0 = 'LOHC_c', bus1 = 'LOHC_b',
             p_nom = dataset_df['Kapazität_Schiff'].max(),
             p_max_pu = dataset_df['Kapazität_Schiff']/dataset_df['Kapazität_Schiff'].max(), 
             p_min_pu = dataset_df['Kapazität_Schiff']/dataset_df['Kapazität_Schiff'].max(),
             p_nom_extendable = True,
             marginal_cost = marginal_cost_shipping_LOHC, 
             capital_cost = capital_cost_shipping_LOHC,
             efficiency = efficiency_shipping_LOHC)

## Links - Pipeline

In [ ]:
network1.add('Link', name = 'Pipeline', bus0 = 'GH2_b', bus1 = 'cologne', 
            p_nom_extendable = True,
            marginal_cost = marginal_cost_pipeline, 
            capital_cost = capital_cost_pipeline,
            lifetime = lifetime_pipeline)

network2.add('Link', name = 'Pipeline', bus0 = 'GH2_b', bus1 = 'cologne', 
            p_nom_extendable = True,
            marginal_cost = marginal_cost_pipeline, 
            capital_cost = capital_cost_pipeline,
            lifetime = lifetime_pipeline)

network3.add('Link', name = 'Pipeline', bus0 = 'GH2_b', bus1 = 'cologne', 
            p_nom_extendable = True,
            marginal_cost = marginal_cost_pipeline, 
            capital_cost = capital_cost_pipeline,
            lifetime = lifetime_pipeline)

## Storages

In [ ]:
network1.add('Store', name = 'Stromspeicher', bus = 'electricity', 
            e_nom_extendable = True,  e_cyclic = True, 
            capital_cost = capital_cost_storage_el,
            marginal_cost = marginal_cost_storage_el,
            standing_loss = standing_loss_el,
            lifetime = lifetime_storage_el)

network2.add('Store', name = 'Stromspeicher', bus = 'electricity', 
            e_nom_extendable = True,  e_cyclic = True, 
            capital_cost = capital_cost_storage_el,
            marginal_cost = marginal_cost_storage_el,
            standing_loss = standing_loss_el,
            lifetime = lifetime_storage_el)

network3.add('Store', name = 'Stromspeicher', bus = 'electricity', 
            e_nom_extendable = True,  e_cyclic = True, 
            capital_cost = capital_cost_storage_el,
            marginal_cost = marginal_cost_storage_el,
            standing_loss = standing_loss_el,
            lifetime = lifetime_storage_el)


network1.add('Store', name = 'Flüssigspeicher Chile', bus = 'LH2_c', 
             e_nom_extendable = True, e_cyclic = True,  
             capital_cost = capital_cost_storage_LH2, 
             marginal_cost = marginal_cost_storage_LH2,
             standing_loss = standing_loss_LH2,
             lifetime = lifetime_storage_LH2)
network1.add('Store', name = 'Flüssigspeicher Bremerhaven', bus = 'LH2_b', 
             e_nom_extendable = True, e_cyclic = True,  
             capital_cost = capital_cost_storage_LH2, 
             marginal_cost = marginal_cost_storage_LH2,
             standing_loss = standing_loss_LH2,
             lifetime = lifetime_storage_LH2)

network2.add('Store', name = 'Ammoniakspeicher Chile', bus = 'NH3_c', 
             e_nom_extendable = True,  e_cyclic = True, 
             capital_cost = capital_cost_storage_NH3, 
             marginal_cost = marginal_cost_storage_NH3,
             standing_loss = standing_loss_NH3,
             lifetime = lifetime_storage_NH3)
network2.add('Store', name = 'Ammoniakspeicher Bremerhaven', bus = 'NH3_b', 
             e_nom_extendable = True,  e_cyclic = True, 
             capital_cost = capital_cost_storage_NH3, 
             marginal_cost = marginal_cost_storage_NH3,
             standing_loss = standing_loss_NH3,
             lifetime = lifetime_storage_NH3)
    
network3.add('Store', name = 'LOHC-Speicher Chile', bus = 'LOHC_c', 
             e_nom_extendable = True, e_cyclic = True,  
             capital_cost = capital_cost_storage_LOHC, 
             marginal_cost = marginal_cost_storage_LOHC,
             standing_loss = standing_loss_LOHC,
             lifetime = lifetime_storage_LOHC)
network3.add('Store', name = 'LOHC-Speicher Bremerhaven', bus = 'LOHC_b', 
             e_nom_extendable = True, e_cyclic = True,  
             capital_cost = capital_cost_storage_LOHC, 
             marginal_cost = marginal_cost_storage_LOHC,
             standing_loss = standing_loss_LOHC,
             lifetime = lifetime_storage_LOHC)

# Grafische Darstellung der Ausgangsprofile

In [ ]:
#Jahresverlauf('Stromerzeugung aus Windkraft',dataset_df.Zeitstempel.loc[2000:2055], dataset_df['Wind_Erzeugung'].loc[2000:2055],'Zeit','Leistung')

In [ ]:
#Jahresverlauf('Stromerzeugung aus Photovoltaik',dataset_df.index, dataset_df['PV_Erzeugung'],'Zeit','Leistung')

In [ ]:
#Jahresverlauf('Elektrolyse',dataset_df.index, dataset_df['Elektrolyse'],'Zeit','Leistung')

In [ ]:
#Jahresverlauf('Ferntransport',dataset_df.index, dataset_df['Kapazität_Schiff'],'Zeit','Leistung')

# Optimierung des Systems

In [ ]:
# Optimierung
network1.lopf(pyomo = False, solver_name = 'gurobi')
network2.lopf(pyomo = False, solver_name = 'gurobi')
network3.lopf(pyomo = False, solver_name = 'gurobi')

In [ ]:
#network1.iplot()

# Auswertung der Ergebnisse

## Ausgabe der Ergebnistabellen (Netzwerk 1)

### Installierte Leistungen, Kapazitäten und Volllststunden

In [ ]:
# Ausgabe einer Ergebnistabelle der optimierten Leistungen in MW und optimierten Kapazitäten in MWh

df_results_1 = pd.DataFrame(network1.generators['p_nom_opt'])
df_results_2 = pd.DataFrame(network1.links['p_nom_opt'])
df_results_3 = pd.DataFrame(network1.stores['e_nom_opt'])

frames = [df_results_1, df_results_2, df_results_3]

results_x = (pd.concat(frames).round(2).fillna(0).rename(columns = {'p_nom_opt':'Leistung [MW]','e_nom_opt':'Speicherkapazität [MWh]'}))
results = results_x.loc[(results_x.sum(axis=1) != 0), (results_x.sum(axis=0) != 0)]
results

# Berechnung der jährlichen Volllaststunden

df_flh_1 = pd.DataFrame((network1.generators_t.p.sum())/network1.generators.p_nom_opt)
df_flh_2 = pd.DataFrame((network1.links_t.p0.sum())/network1.links.p_nom_opt)

df_flh_x = df_flh_1.append(df_flh_2).round(0).fillna(0)
df_flh = df_flh_x.loc[(df_flh_x.sum(axis=1) != 0), (df_flh_x.sum(axis=0) != 0)]
df_flh = df_flh.rename(columns = {0:'Volllast [h]'}).fillna(0)

# Berechnung der Energie 

df_results_1 = pd.DataFrame(network1.generators_t.p.sum())/1000
df_results_2 = pd.DataFrame(-network1.links_t.p1.sum())/1000
df_results_3 = pd.DataFrame(network1.stores_t.e.sum())/1000
frames = [df_results_1, df_results_2, df_results_3]
results_x = (pd.concat(frames).round(2).fillna(0).rename(columns = {0:'Energie [GWh]'}))

results_sum_energy_1 = pd.concat([results,df_flh,results_x],axis=1).fillna(0)
results_sum_energy_1

### Sankey-Diagramm

In [ ]:
#Sankey Diagramm_BErechnungen_LH2

SP_Verluste=(network1.stores.standing_loss*network1.stores_t.e.sum()/1000)
Verluste_Stromspeicher_1=SP_Verluste[0]
Verluste_Speicher_Chile_1=SP_Verluste[1]
Verluste_Speicher_Bremerhaven_1=SP_Verluste[2]

Input_Photovoltaik= results_sum_energy_1['Energie [GWh]']['Photovoltaik']/(results_sum_energy_1['Energie [GWh]']['Photovoltaik']+results_sum_energy_1['Energie [GWh]']['Windkraft'])

Input_Windkraft= results_sum_energy_1['Energie [GWh]']['Windkraft']/(results_sum_energy_1['Energie [GWh]']['Photovoltaik']+results_sum_energy_1['Energie [GWh]']['Windkraft'])


Energie_Stromspeicher= (results_sum_energy_1['Energie [GWh]']['Windkraft']-Verluste_Stromspeicher_1)/(results_sum_energy_1['Energie [GWh]']['Photovoltaik']+results_sum_energy_1['Energie [GWh]']['Windkraft'])

Energie_AEL= ((results_sum_energy_1['Energie [GWh]']['AEL']/(results_sum_energy_1['Energie [GWh]']['Photovoltaik']+results_sum_energy_1['Energie [GWh]']['Windkraft'])))

Energie_Verflüssigung= ((results_sum_energy_1['Energie [GWh]']['Verflüssigung']/(results_sum_energy_1['Energie [GWh]']['Photovoltaik']+results_sum_energy_1['Energie [GWh]']['Windkraft'])))

Energie_Flüssigspeicher_Chile = (results_sum_energy_1['Energie [GWh]']['Verflüssigung']-Verluste_Speicher_Chile_1)/(results_sum_energy_1['Energie [GWh]']['Photovoltaik']+results_sum_energy_1['Energie [GWh]']['Windkraft'])

Energie_Verschiffung= ((results_sum_energy_1['Energie [GWh]']['Verschiffung LH2']/(results_sum_energy_1['Energie [GWh]']['Photovoltaik']+results_sum_energy_1['Energie [GWh]']['Windkraft'])))

Energie_Flüssigspeicher_Bremerhaven= (results_sum_energy_1['Energie [GWh]']['Verschiffung LH2']-Verluste_Speicher_Bremerhaven_1)/(results_sum_energy_1['Energie [GWh]']['Photovoltaik']+results_sum_energy_1['Energie [GWh]']['Windkraft'])

Energie_Regasifizierung= ((results_sum_energy_1['Energie [GWh]']['Regasifizierung']/(results_sum_energy_1['Energie [GWh]']['Photovoltaik']+results_sum_energy_1['Energie [GWh]']['Windkraft'])))

Energie_Pipeline=  ((results_sum_energy_1['Energie [GWh]']['Pipeline']/(results_sum_energy_1['Energie [GWh]']['Photovoltaik']+results_sum_energy_1['Energie [GWh]']['Windkraft'])))

Energie_Last= -(1155/(results_sum_energy_1['Energie [GWh]']['Photovoltaik']+results_sum_energy_1['Energie [GWh]']['Windkraft']))


Verluste_Pipeline=-(Energie_Regasifizierung-Energie_Pipeline)

Verluste_Regasifizierung=-(Energie_Flüssigspeicher_Bremerhaven-Energie_Regasifizierung)
Verluste_Flüssigspeicher_Bremerhaven=-(Energie_Verschiffung-Energie_Flüssigspeicher_Bremerhaven)
Verluste_Verschiffung=-(Energie_Flüssigspeicher_Chile-Energie_Verschiffung)
Verluste_Flüssigspeicher_Chile=-(Energie_Verflüssigung-Energie_Flüssigspeicher_Chile)
Verluste_Verflüssigung=-(Energie_AEL-Energie_Verflüssigung)
Verluste_AEL=-(Energie_Stromspeicher-Energie_AEL)
Verluste_Stromspeicher=-(Input_Windkraft+Input_Photovoltaik-Energie_Stromspeicher)


Sankey_1={'Verluste':[Input_Photovoltaik, Input_Windkraft, Verluste_Stromspeicher, Verluste_AEL,Verluste_Verflüssigung, Verluste_Flüssigspeicher_Chile, Verluste_Verschiffung,Verluste_Flüssigspeicher_Bremerhaven,Verluste_Regasifizierung,Verluste_Pipeline,Energie_Last],
        'Orientierung':[0,0,-1,-1,-1,-1,-1,-1,-1,-1,0],
          'Energie':[Input_Photovoltaik, Input_Windkraft, Energie_Stromspeicher, Energie_AEL,Energie_Verflüssigung, Energie_Flüssigspeicher_Chile, Energie_Verschiffung,Energie_Flüssigspeicher_Bremerhaven,Energie_Regasifizierung,Energie_Pipeline,Energie_Last]}
Verluste_Sankey_1 = pd.DataFrame(data=Sankey_1,
                            index=['Photovoltaik', 'Windkraft', 'Stromspeicher','Elektrolyse','Verflüssigung','Speicher Chile','Verschiffung','Speicher Bremerhaven','Regasifizierung','Pipeline','Last'])
Verluste_Sankey_1.Verluste = Verluste_Sankey_1.Verluste*100
Verluste_Sankey_1.Verluste = Verluste_Sankey_1.Verluste.replace(0, np.nan)
Verluste_Sankey_1 = Verluste_Sankey_1.dropna(axis=0)
Verluste_Sankey_1 = Verluste_Sankey_1.iloc[::-1]
Verluste_Sankey_1

#Verluste_Sankey



In [ ]:
#Sankey Diagramm Verluste

from matplotlib.sankey import Sankey

sankey= Sankey(scale=0.01,offset=0.9,
                format='%.2f', unit='%')
sankey.add(flows=Verluste_Sankey_1.Verluste,labels=Verluste_Sankey_1.index,orientations=Verluste_Sankey_1.Orientierung,rotation=-90)
sankey.finish()
plt.title("Verluste Wasserstoffbereitstellung für Zementfabrik (LH2)")

### Gestehungskosten Wasserstoff

In [ ]:
# Berechnung des Wasserstoffpreises in €/kg

# Kilopreis der Generatoren
Generatorkosten = (((network1.generators.capital_cost * network1.generators.p_nom_opt)/network1.generators.lifetime
                    + network1.generators.marginal_cost * network1.generators_t.p.sum())/hydrogen_demand_kg).fillna(0)
     
Generatorkosten=pd.DataFrame(Generatorkosten,columns=['Preis [€/kg]'])

# Kilopreis der Links
Linkkosten = (((network1.links.p_nom_opt * network1.links.capital_cost)/network1.links.lifetime
              + network1.links.marginal_cost*(-network1.links_t.p1.sum()))
              /hydrogen_demand_kg)

Linkkosten=pd.DataFrame(Linkkosten,columns=['Preis [€/kg]'])

# Kilopreis der Speicher
Speicherkosten = (((network1.stores.e_nom_opt * network1.stores.capital_cost)/network1.stores.lifetime
                   + network1.stores.marginal_cost*network1.stores_t.e.sum())/hydrogen_demand_kg)

Speicherkosten=pd.DataFrame(Speicherkosten,columns=['Preis [€/kg]'])

Kilogrammpreis = (Generatorkosten.append(Linkkosten).append(Speicherkosten))


# Erstellung der Dataframes zur Darstellung in den Kategorien

Kilogrammpreis_Strom = pd.DataFrame((Kilogrammpreis.iloc[[0,1]].append(Kilogrammpreis.loc[['Stromspeicher']]).sum())).rename(index={'Preis [€/kg]': 'Stromkosten'})
Kilogrammpreis_Elektrolyse = pd.DataFrame(Kilogrammpreis.iloc[[2,3]].sum()).rename(index={'Preis [€/kg]': 'Elektrolyse'})
Kilogrammpreis_Umwandlung = pd.DataFrame(Kilogrammpreis.iloc[[4,5]].sum()).rename(index={'Preis [€/kg]': 'Umwandlung'})
Kilogrammpreis_Transport = pd.DataFrame(Kilogrammpreis.iloc[[6,7]].sum()).rename(index={'Preis [€/kg]': 'Transport'})
Kilogrammpreis_Speicherung = pd.DataFrame(Kilogrammpreis.iloc[[9,10]].sum()).rename(index={'Preis [€/kg]': 'Speicherung'})

Gestehungskosten_Wasserstoff_1 = (Kilogrammpreis_Strom
                                .append(Kilogrammpreis_Elektrolyse)
                                .append(Kilogrammpreis_Umwandlung)
                                .append(Kilogrammpreis_Transport)
                                .append(Kilogrammpreis_Speicherung)).rename(columns = {0:'Gestehungspreis [€/kg]'})

In [ ]:
# Ausgabe des Ergebnisses
Gestehungskosten_Wasserstoff_EUR_1 = float(round(Gestehungskosten_Wasserstoff_1.sum(),2))
print('Die Gestehungskosten für ein Kilogramm Wasserstoff liegen bei '+str(Gestehungskosten_Wasserstoff_EUR_1)+' €')

## Ausgabe der Ergebnistabellen (Netzwerk 2)

### Installierte Leistungen, Kapazitäten

In [ ]:
# Ausgabe einer Ergebnistabelle der optimierten Leistungen in MW und optimierten Kapazitäten in MWh

df_results_1 = pd.DataFrame(network2.generators['p_nom_opt'])
df_results_2 = pd.DataFrame(network2.links['p_nom_opt'])
df_results_3 = pd.DataFrame(network2.stores['e_nom_opt'])

frames = [df_results_1, df_results_2, df_results_3]

results_x = (pd.concat(frames).round(2).fillna(0).rename(columns = {'p_nom_opt':'Leistung [MW]','e_nom_opt':'Speicherkapazität [MWh]'}))
results = results_x.loc[(results_x.sum(axis=1) != 0), (results_x.sum(axis=0) != 0)]
results

# Berechnung der jährlichen Volllaststunden

df_flh_1 = pd.DataFrame((network2.generators_t.p.sum())/network2.generators.p_nom_opt)
df_flh_2 = pd.DataFrame((network2.links_t.p0.sum())/network2.links.p_nom_opt)

df_flh_x = df_flh_1.append(df_flh_2).round(0).fillna(0)
df_flh = df_flh_x.loc[(df_flh_x.sum(axis=1) != 0), (df_flh_x.sum(axis=0) != 0)]
df_flh = df_flh.rename(columns = {0:'Volllast [h]'}).fillna(0)
#Berechnung der Energie 
df_results_1 = pd.DataFrame(network2.generators_t.p.sum())/1000
df_results_2 = pd.DataFrame(-network2.links_t.p1.sum())/1000
df_results_3 = pd.DataFrame(network2.stores_t.e.sum())/1000
frames = [df_results_1, df_results_2, df_results_3]
results_x = (pd.concat(frames).round(2).fillna(0).rename(columns = {0:'Energie [GWh]'}))

results_sum_energy_2 = pd.concat([results,df_flh,results_x],axis=1).fillna(0)
results_sum_energy_2

### Sankey-Diagramm

In [ ]:
#Sankey Diagramm_BErechnungen_NH3
SP_Verluste_2=(network2.stores.standing_loss*network2.stores_t.e.sum()/1000)
Verluste_Stromspeicher_2=SP_Verluste_2[0]
Verluste_Speicher_Chile_2=SP_Verluste_2[1]
Verluste_Speicher_Bremerhaven_2=SP_Verluste_2[2]

Input_Photovoltaik= results_sum_energy_2['Energie [GWh]']['Photovoltaik']/(results_sum_energy_2['Energie [GWh]']['Photovoltaik']+results_sum_energy_2['Energie [GWh]']['Windkraft'])

Input_Windkraft= results_sum_energy_2['Energie [GWh]']['Windkraft']/(results_sum_energy_2['Energie [GWh]']['Photovoltaik']+results_sum_energy_2['Energie [GWh]']['Windkraft'])


Energie_Stromspeicher= (results_sum_energy_2['Energie [GWh]']['Windkraft']-Verluste_Stromspeicher_2)/(results_sum_energy_2['Energie [GWh]']['Photovoltaik']+results_sum_energy_2['Energie [GWh]']['Windkraft'])

Energie_AEL= ((results_sum_energy_2['Energie [GWh]']['AEL']/(results_sum_energy_2['Energie [GWh]']['Photovoltaik']+results_sum_energy_2['Energie [GWh]']['Windkraft'])))

Energie_Verflüssigung= ((results_sum_energy_2['Energie [GWh]']['Haber-Bosch']/(results_sum_energy_2['Energie [GWh]']['Photovoltaik']+results_sum_energy_2['Energie [GWh]']['Windkraft'])))

Energie_Flüssigspeicher_Chile = (results_sum_energy_2['Energie [GWh]']['Haber-Bosch']-Verluste_Speicher_Chile_2)/(results_sum_energy_2['Energie [GWh]']['Photovoltaik']+results_sum_energy_2['Energie [GWh]']['Windkraft'])

Energie_Verschiffung= ((results_sum_energy_2['Energie [GWh]']['Verschiffung NH3']/(results_sum_energy_2['Energie [GWh]']['Photovoltaik']+results_sum_energy_2['Energie [GWh]']['Windkraft'])))

Energie_Flüssigspeicher_Bremerhaven= (results_sum_energy_2['Energie [GWh]']['Verschiffung NH3']-Verluste_Speicher_Bremerhaven_2)/(results_sum_energy_2['Energie [GWh]']['Photovoltaik']+results_sum_energy_2['Energie [GWh]']['Windkraft'])

Energie_Regasifizierung= ((results_sum_energy_2['Energie [GWh]']['NH3 Cracker']/(results_sum_energy_2['Energie [GWh]']['Photovoltaik']+results_sum_energy_2['Energie [GWh]']['Windkraft'])))

Energie_Pipeline=  ((results_sum_energy_2['Energie [GWh]']['Pipeline']/(results_sum_energy_2['Energie [GWh]']['Photovoltaik']+results_sum_energy_2['Energie [GWh]']['Windkraft'])))

Energie_Last= -(1155/(results_sum_energy_2['Energie [GWh]']['Photovoltaik']+results_sum_energy_2['Energie [GWh]']['Windkraft']))


Verluste_Pipeline=-(Energie_Regasifizierung-Energie_Pipeline)

Verluste_Regasifizierung=-(Energie_Flüssigspeicher_Bremerhaven-Energie_Regasifizierung)
Verluste_Flüssigspeicher_Bremerhaven=-(Energie_Verschiffung-Energie_Flüssigspeicher_Bremerhaven)
Verluste_Verschiffung=-(Energie_Flüssigspeicher_Chile-Energie_Verschiffung)
Verluste_Flüssigspeicher_Chile=-(Energie_Verflüssigung-Energie_Flüssigspeicher_Chile)
Verluste_Verflüssigung=-(Energie_AEL-Energie_Verflüssigung)
Verluste_AEL=-(Energie_Stromspeicher-Energie_AEL)
Verluste_Stromspeicher=-(Input_Windkraft+Input_Photovoltaik-Energie_Stromspeicher)


Sankey_2={'Verluste':[Input_Photovoltaik, Input_Windkraft, Verluste_Stromspeicher, Verluste_AEL,Verluste_Verflüssigung, Verluste_Flüssigspeicher_Chile, Verluste_Verschiffung,Verluste_Flüssigspeicher_Bremerhaven,Verluste_Regasifizierung,Verluste_Pipeline,Energie_Last],
        'Orientierung':[0,0,-1,-1,-1,-1,-1,-1,-1,-1,0],
          'Energie':[Input_Photovoltaik, Input_Windkraft, Energie_Stromspeicher, Energie_AEL,Energie_Verflüssigung, Energie_Flüssigspeicher_Chile, Energie_Verschiffung,Energie_Flüssigspeicher_Bremerhaven,Energie_Regasifizierung,Energie_Pipeline,Energie_Last]}
Verluste_Sankey_2 = pd.DataFrame(data=Sankey_2,
                            index=['Photovoltaik', 'Windkraft', 'Stromspeicher','Elektrolyse','Verflüssigung','Speicher Chile','Verschiffung','Speicher Bremerhaven','Regasifizierung','Pipeline','Last'])
Verluste_Sankey_2.Verluste = Verluste_Sankey_2.Verluste*100
Verluste_Sankey_2.Verluste = Verluste_Sankey_2.Verluste.replace(0, np.nan)
Verluste_Sankey_2 = Verluste_Sankey_2.dropna(axis=0)
Verluste_Sankey_2 = Verluste_Sankey_2.iloc[::-1]
Verluste_Sankey_2
#Verluste_Sankey_2



In [ ]:
#Sankey Diagramm Verluste

from matplotlib.sankey import Sankey

sankey= Sankey(scale=0.01, offset=0.7,
                format='%.2f', unit='%')
sankey.add(flows=Verluste_Sankey_2.Verluste,labels=Verluste_Sankey_2.index,orientations=Verluste_Sankey_2.Orientierung,rotation=-90)
sankey.finish()
plt.title("Energetische Verluste Wasserstoffbereitstellung für Zementfabrik (NH3)")

### Gestehungskosten Wasserstoff

In [ ]:
# Berechnung des Wasserstoffpreises in €/kg

# Kilopreis der Generatoren
Generatorkosten = (((network2.generators.capital_cost * network2.generators.p_nom_opt)/network2.generators.lifetime
                    + network2.generators.marginal_cost * network2.generators_t.p.sum())/hydrogen_demand_kg).fillna(0)
     
Generatorkosten=pd.DataFrame(Generatorkosten,columns=['Preis [€/kg]'])

# Kilopreis der Links
Linkkosten = (((network2.links.p_nom_opt * network2.links.capital_cost)/network2.links.lifetime
              + network2.links.marginal_cost*(-network2.links_t.p1.sum()))
              /hydrogen_demand_kg)

Linkkosten=pd.DataFrame(Linkkosten,columns=['Preis [€/kg]'])

# Kilopreis der Speicher
Speicherkosten = (((network2.stores.e_nom_opt * network2.stores.capital_cost)/network2.stores.lifetime
                   + network2.stores.marginal_cost*network2.stores_t.e.sum())/hydrogen_demand_kg)

Speicherkosten=pd.DataFrame(Speicherkosten,columns=['Preis [€/kg]'])

Kilogrammpreis = (Generatorkosten.append(Linkkosten).append(Speicherkosten))

# Erstellung der Dataframes zur Darstellung in den Kategorien

Kilogrammpreis_Strom = pd.DataFrame((Kilogrammpreis.iloc[[0,1]].append(Kilogrammpreis.loc[['Stromspeicher']]).sum())).rename(index={'Preis [€/kg]': 'Stromkosten'})
Kilogrammpreis_Elektrolyse = pd.DataFrame(Kilogrammpreis.iloc[[2,3]].sum()).rename(index={'Preis [€/kg]': 'Elektrolyse'})
Kilogrammpreis_Umwandlung = pd.DataFrame(Kilogrammpreis.iloc[[4,5]].sum()).rename(index={'Preis [€/kg]': 'Umwandlung'})
Kilogrammpreis_Transport = pd.DataFrame(Kilogrammpreis.iloc[[6,7]].sum()).rename(index={'Preis [€/kg]': 'Transport'})
Kilogrammpreis_Speicherung = pd.DataFrame(Kilogrammpreis.iloc[[9,10]].sum()).rename(index={'Preis [€/kg]': 'Speicherung'})

Gestehungskosten_Wasserstoff_2 = (Kilogrammpreis_Strom
                                .append(Kilogrammpreis_Elektrolyse)
                                .append(Kilogrammpreis_Umwandlung)
                                .append(Kilogrammpreis_Transport)
                                .append(Kilogrammpreis_Speicherung)).rename(columns = {0:'Gestehungspreis [€/kg]'})

#Kilogrammpreis
#Gestehungskosten_Wasserstoff_2

In [ ]:
# Ausgabe des Ergebnisses
Gestehungskosten_Wasserstoff_EUR_2 = float(round(Gestehungskosten_Wasserstoff_2.sum(),2))
print('Die Gestehungskosten für ein Kilogramm Wasserstoff liegen bei '+str(Gestehungskosten_Wasserstoff_EUR_2)+' €')

## Ausgabe der Ergebnistabellen (Netzwerk 3)

### Installierte Leistungen, Kapazitäten

In [ ]:
# Ausgabe einer Ergebnistabelle der optimierten Leistungen in MW und optimierten Kapazitäten in MWh

df_results_1 = pd.DataFrame(network3.generators['p_nom_opt'])
df_results_2 = pd.DataFrame(network3.links['p_nom_opt'])
df_results_3 = pd.DataFrame(network3.stores['e_nom_opt'])

frames = [df_results_1, df_results_2, df_results_3]

results_x = (pd.concat(frames).round(2).fillna(0).rename(columns = {'p_nom_opt':'Leistung [MW]','e_nom_opt':'Speicherkapazität [MWh]'}))
results = results_x.loc[(results_x.sum(axis=1) != 0), (results_x.sum(axis=0) != 0)]
results

# Berechnung der jährlichen Volllaststunden

df_flh_1 = pd.DataFrame((network3.generators_t.p.sum())/network3.generators.p_nom_opt)
df_flh_2 = pd.DataFrame((network3.links_t.p0.sum())/network3.links.p_nom_opt)

df_flh_x = df_flh_1.append(df_flh_2).round(0).fillna(0)
df_flh = df_flh_x.loc[(df_flh_x.sum(axis=1) != 0), (df_flh_x.sum(axis=0) != 0)]
df_flh = df_flh.rename(columns = {0:'Volllast [h]'}).fillna(0)
#Berechnung der Energie 
df_results_1 = pd.DataFrame(network3.generators_t.p.sum())/1000
df_results_2 = pd.DataFrame(-network3.links_t.p1.sum())/1000
df_results_3 = pd.DataFrame(network3.stores_t.e.sum())/1000
frames = [df_results_1, df_results_2, df_results_3]
results_x = (pd.concat(frames).round(2).fillna(0).rename(columns = {0:'Energie [GWh]'}))
#df_flh

results_sum_energy_3 = pd.concat([results,df_flh,results_x],axis=1).fillna(0)
results_sum_energy_3

### Sankey-Diagramm

In [ ]:
#Sankey Diagramm_BErechnungen_LOHC

SP_Verluste_3=(network3.stores.standing_loss*network3.stores_t.e.sum()/1000)
Verluste_Stromspeicher_3=SP_Verluste_3[0]
Verluste_Speicher_Chile_3=SP_Verluste_3[1]
Verluste_Speicher_Bremerhaven_3=SP_Verluste_3[2]

Input_Photovoltaik= results_sum_energy_3['Energie [GWh]']['Photovoltaik']/(results_sum_energy_3['Energie [GWh]']['Photovoltaik']+results_sum_energy_3['Energie [GWh]']['Windkraft'])

Input_Windkraft= results_sum_energy_3['Energie [GWh]']['Windkraft']/(results_sum_energy_3['Energie [GWh]']['Photovoltaik']+results_sum_energy_3['Energie [GWh]']['Windkraft'])


Energie_Stromspeicher= (results_sum_energy_3['Energie [GWh]']['Windkraft']-Verluste_Stromspeicher_3)/(results_sum_energy_3['Energie [GWh]']['Photovoltaik']+results_sum_energy_3['Energie [GWh]']['Windkraft'])

Energie_AEL= ((results_sum_energy_3['Energie [GWh]']['AEL']/(results_sum_energy_3['Energie [GWh]']['Photovoltaik']+results_sum_energy_3['Energie [GWh]']['Windkraft'])))

Energie_Verflüssigung= ((results_sum_energy_3['Energie [GWh]']['Hydrierung']/(results_sum_energy_3['Energie [GWh]']['Photovoltaik']+results_sum_energy_3['Energie [GWh]']['Windkraft'])))

Energie_Flüssigspeicher_Chile = (results_sum_energy_3['Energie [GWh]']['Hydrierung']-Verluste_Speicher_Chile_3)/(results_sum_energy_3['Energie [GWh]']['Photovoltaik']+results_sum_energy_3['Energie [GWh]']['Windkraft'])

Energie_Verschiffung= ((results_sum_energy_3['Energie [GWh]']['Verschiffung LOHC']/(results_sum_energy_3['Energie [GWh]']['Photovoltaik']+results_sum_energy_3['Energie [GWh]']['Windkraft'])))

Energie_Flüssigspeicher_Bremerhaven= (results_sum_energy_3['Energie [GWh]']['Verschiffung LOHC']-Verluste_Speicher_Bremerhaven_3)/(results_sum_energy_3['Energie [GWh]']['Photovoltaik']+results_sum_energy_3['Energie [GWh]']['Windkraft'])

Energie_Regasifizierung= ((results_sum_energy_3['Energie [GWh]']['Dehydrierung']/(results_sum_energy_3['Energie [GWh]']['Photovoltaik']+results_sum_energy_3['Energie [GWh]']['Windkraft'])))

Energie_Pipeline=  ((results_sum_energy_3['Energie [GWh]']['Pipeline']/(results_sum_energy_3['Energie [GWh]']['Photovoltaik']+results_sum_energy_3['Energie [GWh]']['Windkraft'])))

Energie_Last= -(1155/(results_sum_energy_3['Energie [GWh]']['Photovoltaik']+results_sum_energy_3['Energie [GWh]']['Windkraft']))


Verluste_Pipeline=-(Energie_Regasifizierung-Energie_Pipeline)

Verluste_Regasifizierung=-(Energie_Flüssigspeicher_Bremerhaven-Energie_Regasifizierung)
Verluste_Flüssigspeicher_Bremerhaven=-(Energie_Verschiffung-Energie_Flüssigspeicher_Bremerhaven)
Verluste_Verschiffung=-(Energie_Flüssigspeicher_Chile-Energie_Verschiffung)
Verluste_Flüssigspeicher_Chile=-(Energie_Verflüssigung-Energie_Flüssigspeicher_Chile)
Verluste_Verflüssigung=-(Energie_AEL-Energie_Verflüssigung)
Verluste_AEL=-(Energie_Stromspeicher-Energie_AEL)
Verluste_Stromspeicher=-(Input_Windkraft+Input_Photovoltaik-Energie_Stromspeicher)


Sankey_3={'Verluste':[Input_Photovoltaik, Input_Windkraft, Verluste_Stromspeicher, Verluste_AEL,Verluste_Verflüssigung, Verluste_Flüssigspeicher_Chile, Verluste_Verschiffung,Verluste_Flüssigspeicher_Bremerhaven,Verluste_Regasifizierung,Verluste_Pipeline,Energie_Last],
        'Orientierung':[0,0,-1,-1,-1,-1,-1,-1,-1,-1,0],
          'Energie':[Input_Photovoltaik, Input_Windkraft, Energie_Stromspeicher, Energie_AEL,Energie_Verflüssigung, Energie_Flüssigspeicher_Chile, Energie_Verschiffung,Energie_Flüssigspeicher_Bremerhaven,Energie_Regasifizierung,Energie_Pipeline,Energie_Last]}
Verluste_Sankey_3 = pd.DataFrame(data=Sankey_3,
                            index=['Photovoltaik', 'Windkraft', 'Stromspeicher','Elektrolyse','Verflüssigung','Speicher Chile','Verschiffung','Speicher Bremerhaven','Regasifizierung','Pipeline','Last'])
Verluste_Sankey_3.Verluste = Verluste_Sankey_3.Verluste*100
Verluste_Sankey_3.Verluste = Verluste_Sankey_3.Verluste.replace(0, np.nan)
Verluste_Sankey_3 = Verluste_Sankey_3.dropna(axis=0)
Verluste_Sankey_3 = Verluste_Sankey_3.iloc[::-1]
Verluste_Sankey_3


In [ ]:
#Sankey Diagramm Verluste

from matplotlib.sankey import Sankey

sankey= Sankey(scale=0.01, offset=0.7,
                format='%.2f', unit='%')
sankey.add(flows=Verluste_Sankey_3.Verluste,labels=Verluste_Sankey_3.index,orientations=Verluste_Sankey_3.Orientierung,rotation=-90)
sankey.finish()
plt.title("Energetische Verluste Wasserstoffbereitstellung für Zementfabrik (LOHC)")

### Gestehungskosten Wasserstoff

In [ ]:
# Berechnung des Wasserstoffpreises in €/kg

# Kilopreis der Generatoren
Generatorkosten = (((network3.generators.capital_cost * network3.generators.p_nom_opt)/network3.generators.lifetime
                    + network3.generators.marginal_cost * network3.generators_t.p.sum())/hydrogen_demand_kg).fillna(0)
     
Generatorkosten=pd.DataFrame(Generatorkosten,columns=['Preis [€/kg]'])

# Kilopreis der Links
Linkkosten = (((network3.links.p_nom_opt * network3.links.capital_cost)/network3.links.lifetime
              + network3.links.marginal_cost*(-network3.links_t.p1.sum()))
              /hydrogen_demand_kg)

Linkkosten=pd.DataFrame(Linkkosten,columns=['Preis [€/kg]'])

# Kilopreis der Speicher
Speicherkosten = (((network3.stores.e_nom_opt * network3.stores.capital_cost)/network3.stores.lifetime
                   + network3.stores.marginal_cost*network3.stores_t.e.sum())/hydrogen_demand_kg)

Speicherkosten=pd.DataFrame(Speicherkosten,columns=['Preis [€/kg]'])


Kilogrammpreis = (Generatorkosten.append(Linkkosten).append(Speicherkosten))

# Erstellung der Dataframes zur Darstellung in den Kategorien
Kilogrammpreis_Strom = pd.DataFrame((Kilogrammpreis.iloc[[0,1]].append(Kilogrammpreis.loc[['Stromspeicher']]).sum())).rename(index={'Preis [€/kg]': 'Stromkosten'})
Kilogrammpreis_Elektrolyse = pd.DataFrame(Kilogrammpreis.iloc[[2,3]].sum()).rename(index={'Preis [€/kg]': 'Elektrolyse'})
Kilogrammpreis_Umwandlung = pd.DataFrame(Kilogrammpreis.iloc[[4,5]].sum()).rename(index={'Preis [€/kg]': 'Umwandlung'})
Kilogrammpreis_Transport = pd.DataFrame(Kilogrammpreis.iloc[[6,7]].sum()).rename(index={'Preis [€/kg]': 'Transport'})
Kilogrammpreis_Speicherung = pd.DataFrame(Kilogrammpreis.iloc[[9,10]].sum()).rename(index={'Preis [€/kg]': 'Speicherung'})

Gestehungskosten_Wasserstoff_3 = (Kilogrammpreis_Strom
                                .append(Kilogrammpreis_Elektrolyse)
                                .append(Kilogrammpreis_Umwandlung)
                                .append(Kilogrammpreis_Transport)
                                .append(Kilogrammpreis_Speicherung)).rename(columns = {0:'Gestehungspreis [€/kg]'})

In [ ]:
# Ausgabe des Ergebnisses
Gestehungskosten_Wasserstoff_EUR_3 = float(round(Gestehungskosten_Wasserstoff_3.sum(),2))
print('Die Gestehungskosten für ein Kilogramm Wasserstoff liegen bei '+str(Gestehungskosten_Wasserstoff_EUR_3)+' €')

## Ergebnisübersicht

In [ ]:
# Ergebnisse

results = [Gestehungskosten_Wasserstoff_1,Gestehungskosten_Wasserstoff_2,Gestehungskosten_Wasserstoff_3]

results_sum = pd.concat(results, axis = 1)
results_sum

## Grafische Darstellungen

### Darstellung der optimierten Erzeugungsleistung

In [ ]:
Jahresverlauf('EE-Erzeugungsleistung','2025-01-01','2025-12-31',(network1.generators_t.p),'Stunde','Zeit','Leistung [MW]',1,1)

In [ ]:
Jahresverlauf('EE-Erzeugungsleistung','2025-01-01','2025-12-31',(network2.generators_t.p),'Stunde','Zeit','Leistung [MW]',1,1)

In [ ]:
Jahresverlauf('EE-Erzeugungsleistung','2025-01-01','2025-12-31',(network3.generators_t.p),'Stunde','Zeit','Leistung [MW]',1,1)

In [ ]:
data_links_1 = network1.links_t.p0['AEL']
data_links_1['Verflüssigung'] = network1.links_t.p0['Verflüssigung']
data_links_1['Regasifizierung'] = network1.links_t.p0['Regasifizierung']

In [ ]:
Jahresverlauf('Umwandlungsleistung','2025-01-01','2025-01-31',(data_links_1),'Stunde','Zeit','Leistung [MW]',1,1)

In [ ]:
# Plot

data = network1.generators_t.p

ax = network1.generators_t.p.plot()

plt.axhline(y=network1.generators_t.p['Photovoltaik'].max(), xmin=0.01, xmax=0.99, color = 'black', linestyle = "--")
plt.axhline(y=network1.generators_t.p['Windkraft'].max(), xmin=0.01, xmax=0.99, color = 'black', linestyle = "--")

plt.text(s = 'Photovoltaik: '+str(network1.generators.p_nom_opt.round(0)['Photovoltaik'])+' MW' , y = network1.generators_t.p['Photovoltaik'].max() + 10, x = 0, color = 'black', fontsize = 12)
plt.text(s = 'Windkraft: '+str(network1.generators.p_nom_opt.round(0)['Windkraft'])+' MW', y = network1.generators_t.p['Windkraft'].max() + 10, x = 0, color = 'black', fontsize = 12)

ax.set_title('Erzeugungsleistung')
ax.set_ylabel("Leistung [MW]")
horiz_offset = 1.03
vert_offset = 1.
ax.legend(bbox_to_anchor=(horiz_offset, vert_offset))
#plt.show()                  

### Darstellung der optimierten Erzeugungsleistung als Jahresdauerline

In [ ]:
if location_north:
    Dauerlinie(network2.generators_t.p,'Photovoltaik','Jahresdauerlinie EE-Erzeugung','Leistung [MW]')
else:
    Dauerlinie(network2.generators_t.p,'Windkraft','Jahresdauerlinie EE-Erzeugung','Leistung  [MW]')

In [ ]:
Dauerlinie(network2.links_t.p0,'AEL','Jahresdauerlinie Elektrolyse','Leistung  [MW]')

### Darstellung der optimierten Elektrolyseleistung

In [ ]:
# Darstellung der optimierten Elektrolyseleistung

fig, ax = plt.subplots()

ax.plot(network1.links_t.p0['PEMEL'])
ax.plot(network1.links_t.p0['AEL'])
#ax.plot(network2.links_t.p0['AEL'])
#ax.plot(network3.links_t.p0['AEL'])

plt.axhline(y=network1.links_t.p0['PEMEL'].max(), xmin=0.01, xmax=0.99, color = 'black', linestyle = "--")
plt.axhline(y=network1.links_t.p0['AEL'].max(), xmin=0.01, xmax=0.99, color = 'black', linestyle = "--")

plt.text(s = 'PEMEL: '+str(network1.links.p_nom_opt.round(0)['PEMEL'])+' MW' , y = network1.links_t.p0['PEMEL'].max() + 10, x = 0, color = 'black', fontsize = 12)
plt.text(s = 'AEL: '+str(network1.links.p_nom_opt.round(0)['AEL'])+' MW', y = network1.links_t.p0['AEL'].max() + 10, x = 0, color = 'black', fontsize = 12)

ax.set_title('Elektrolyseleistung')
ax.set_ylabel("Leistung [MW]")
plt.show()         

### Darstellung der Bereitstellungskosten

In [ ]:
# Übersicht der Kilopreise

labels = ['LH2', 'NH3', 'LOHC']
Stromkosten = np.array([Gestehungskosten_Wasserstoff_1['Gestehungspreis [€/kg]']['Stromkosten'],
                        Gestehungskosten_Wasserstoff_2['Gestehungspreis [€/kg]']['Stromkosten'],
                         Gestehungskosten_Wasserstoff_3['Gestehungspreis [€/kg]']['Stromkosten']])

Elektrolyse = np.array([Gestehungskosten_Wasserstoff_1['Gestehungspreis [€/kg]']['Elektrolyse'],
                        Gestehungskosten_Wasserstoff_2['Gestehungspreis [€/kg]']['Elektrolyse'],
                         Gestehungskosten_Wasserstoff_3['Gestehungspreis [€/kg]']['Elektrolyse']])
                         
Umwandlung =  np.array([Gestehungskosten_Wasserstoff_1['Gestehungspreis [€/kg]']['Umwandlung'],
                        Gestehungskosten_Wasserstoff_2['Gestehungspreis [€/kg]']['Umwandlung'],
                         Gestehungskosten_Wasserstoff_3['Gestehungspreis [€/kg]']['Umwandlung']])
                          
Ferntransport_1 = np.array([Gestehungskosten_Wasserstoff_1['Gestehungspreis [€/kg]']['Transport'],
                        Gestehungskosten_Wasserstoff_2['Gestehungspreis [€/kg]']['Transport'],
                         Gestehungskosten_Wasserstoff_3['Gestehungspreis [€/kg]']['Transport']])

                         
Speicherung = np.array([Gestehungskosten_Wasserstoff_1['Gestehungspreis [€/kg]']['Speicherung'],
                        Gestehungskosten_Wasserstoff_2['Gestehungspreis [€/kg]']['Speicherung'],
                         Gestehungskosten_Wasserstoff_3['Gestehungspreis [€/kg]']['Speicherung']])
# the width of the bars: can also be len(x) sequence
width=0.5
fig, ax = plt.subplots(figsize=(14,8))
#x_pos = [-2,0,2]
ax.bar(labels, Stromkosten, label='Stromkosten',width=width,color='lightcoral')
ax.bar(labels, Elektrolyse, bottom=Stromkosten,
       label='Elektrolyse',width=width,color='indianred')
ax.bar(labels, Speicherung, bottom=Stromkosten+Elektrolyse,
       label='Speicherung',width=width,color='firebrick')
ax.bar(labels, Ferntransport_1, bottom=Stromkosten+Elektrolyse+Speicherung,
       label='Ferntransport',width=width,color='brown')
ax.bar(labels, Umwandlung, bottom=Stromkosten+Elektrolyse+Speicherung+Ferntransport_1,
       label='Umwandlung',width=width,color='darkred')

ax.set_ylabel('Bereitstellungskosten in [€/kgH2]')
ax.set_title('Vergleich der Transportmedien')
ax.legend(loc='upper right')

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.text(x+width/2, 
            y+height/2, 
            '{:.02f}€'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',
            color='white',
            fontsize=8,
            )
  

horiz_offset = 1.18
vert_offset = 1.015
ax.legend(bbox_to_anchor=(horiz_offset, vert_offset))
#plt.xticks(x_pos, labels)
plt.show()

## Vergleich der Standorte

In [ ]:
# Ausgabe und Einlesen der Ergebnisse als Excel

if location_north:
    results_sum.to_excel('results_sum_north.xlsx')
else:
    results_sum.to_excel('results_sum_south.xlsx')
    
results_north = pd.read_excel('results_sum_north.xlsx', header = 0, index_col = 0)
results_south = pd.read_excel('results_sum_south.xlsx', header = 0, index_col = 0)    
    
results_south_LH2 = results_south['Gestehungspreis [€/kg]']
results_south_NH3 = results_south['Gestehungspreis [€/kg].1']
results_south_LOHC = results_south['Gestehungspreis [€/kg].2']

results_north_LH2 = results_north['Gestehungspreis [€/kg]']
results_north_NH3 = results_north['Gestehungspreis [€/kg].1']
results_north_LOHC = results_north['Gestehungspreis [€/kg].2']
    

In [ ]:
# Plot

labels = ['Antofagasta', 'Punta Arenas']
Stromkosten = np.array([results_north_NH3.iloc[0],
                        results_south_NH3.iloc[0]])

Elektrolyse = np.array([results_north_NH3.iloc[1],
                        results_south_NH3.iloc[1]])
                         
Umwandlung =  np.array([results_north_NH3.iloc[2],
                        results_north_NH3.iloc[2]])
                          
Ferntransport_1 = np.array([results_north_NH3.iloc[3],
                        results_south_NH3.iloc[3]])

                         
Speicherung = np.array([results_north_NH3.iloc[4],
                        results_south_NH3.iloc[4]])

# the width of the bars: can also be len(x) sequence
width=0.5
fig, ax = plt.subplots(figsize=(14,8))
#x_pos = [-2,0,2]
ax.bar(labels, Stromkosten, label='Stromkosten',width=width,color='lightcoral')
ax.bar(labels, Elektrolyse, bottom=Stromkosten,
       label='Elektrolyse',width=width,color='indianred')
ax.bar(labels, Speicherung, bottom=Stromkosten+Elektrolyse,
       label='Speicherung',width=width,color='firebrick')
ax.bar(labels, Ferntransport_1, bottom=Stromkosten+Elektrolyse+Speicherung,
       label='Ferntransport',width=width,color='brown')
ax.bar(labels, Umwandlung, bottom=Stromkosten+Elektrolyse+Speicherung+Ferntransport_1,
       label='Umwandlung',width=width,color='darkred')

ax.set_ylabel('Bereitstellungskosten in [€/kgH2]')
ax.set_title('Vergleich der Standorte')
ax.legend(loc='upper right')

for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.text(x+width/2, 
            y+height/2, 
            '{:.02f}€'.format(height), 
            horizontalalignment='center', 
            verticalalignment='center',
            color='white',
            fontsize=8,
            )
  

horiz_offset = 1.18
vert_offset = 1.015
ax.legend(bbox_to_anchor=(horiz_offset, vert_offset))
#plt.xticks(x_pos, labels)
plt.show()

# Werkstatt

In [ ]:
# Dataframes weitere Abbildungen
data_speicher_1 = network1.stores_t.e.drop(columns=['Stromspeicher'])/hydrogen_energy_content
data_speicher_2 = network2.stores_t.e.drop(columns=['Stromspeicher'])/hydrogen_energy_content
data_speicher_3 = network3.stores_t.e.drop(columns=['Stromspeicher'])/hydrogen_energy_content

data_transport_1 = (network1.stores_t.e.drop(columns=['Stromspeicher'])/hydrogen_energy_content)
data_transport_1['Verschiffung LH2'] = (network1.links_t.p0['Verschiffung LH2']/hydrogen_energy_content*380)

data_transport_2 = (network2.stores_t.e.drop(columns=['Stromspeicher'])/hydrogen_energy_content)
data_transport_2['Verschiffung NH3'] = (network2.links_t.p0['Verschiffung NH3']/hydrogen_energy_content*380)

data_transport_3 = (network3.stores_t.e.drop(columns=['Stromspeicher'])/hydrogen_energy_content)
data_transport_3['Verschiffung LOHC'] = (network3.links_t.p0['Verschiffung LOHC']/hydrogen_energy_content*380)


In [ ]:
# Speicher LH2
Jahresverlauf('Füllstände der Speicher','2025-01-01','2025-12-31',(data_speicher_1),'Stunde','Zeit','Kapazität [t]',1,1)

In [ ]:
# Speicher NH3
Jahresverlauf('Füllstände der Speicher','2025-01-01','2025-12-31',(data_speicher_2),'Stunde','Zeit','Kapazität [t]',1,1)

In [ ]:
# Speicher LOHC
Jahresverlauf('Füllstände der Speicher','2025-01-01','2025-12-31',(data_speicher_3),'Stunde','Zeit','Kapazität [t]',1,1)

### Vergleich von Speicherung und Ferntransport

In [ ]:
# Plot

a=0
b=8760/9*2
fig, ax1 = plt.subplots(sharey = True)
Verschiffung_Tonnen=network2.links_t.p0['Verschiffung NH3']/hydrogen_energy_content
ax2 = ax1.twinx()
ax2.plot(Verschiffung_Tonnen.loc[a:b],
        label = 'Verschiffung',color='red')
ax1.plot(network2.stores_t.e['Ammoniakspeicher Chile'].loc[a:b]/hydrogen_energy_content,
        label = 'Ammoniakspeicher Chile')
ax1.plot(network2.stores_t.e['Ammoniakspeicher Bremerhaven'].loc[a:b]/hydrogen_energy_content,
        label = 'Ammoniakspeicher Bremerhaven')
ax2.set_ylabel('Verschiffung [t/h]', color = 'red')
ax1.set_ylabel('Kapazität Speicher [t]')
ax1.set_xlabel('Zeit in [h]')


horiz_offset = 1.1
vert_offset = 0.89
ax1.set_title('Beladung des Schiffes')
fig.legend(bbox_to_anchor=(horiz_offset, vert_offset))
plt.show()